# Convolutional neural network

## Load X,y data from NPZ

Using the function added to *mylib.py* file, it's now easy to grab data and X/y vectors ready to be used for model training and tuning

In [1]:
# Run content of mylib.py file
%run mylib.py

# Load data from NPZ file
#data=loadNpz()
(data, X, y)=loadXy(verbose=False)

https://www.tutorialspoint.com/how-can-keras-be-used-to-extract-features-from-only-one-layer-of-the-model-using-python

In [2]:
import numpy as np


IMAGE_SIZE=data['IMAGE_SIZE']

IMAGE_COLOR_SPACE_DIM=data['IMAGE_COLOR_SPACE_DIM']

NB_LABELS=data['IMAGE_NB_LABELS']

FEATURE_SIZE=data['FEATURE_SIZE']

DATASET_NAME=data['DATASET_NAME']


KERAS_INPUT_SHAPE=(IMAGE_SIZE[0], IMAGE_SIZE[1], IMAGE_COLOR_SPACE_DIM)


X_tr=np.float32(data['trainX']['data'])
y_tr=data['trainX']['labels']

X_te=np.float32(data['test']['data'])
y_te=data['test']['labels']


In [22]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras as keras
from tensorflow.keras import activations

# Convolutional Network
model = keras.Sequential()
model.add(keras.layers.Conv2D(filters=32, kernel_size=5, strides=2,
                              activation='relu', input_shape=KERAS_INPUT_SHAPE))
model.add(keras.layers.MaxPool2D(pool_size=3))
model.add(keras.layers.Conv2D(filters=32, kernel_size=3, strides=1,
                              activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=2048, activation='relu', name='high-level-features'))
model.add(keras.layers.Dense(units=512, activation='relu'))
model.add(keras.layers.Dense(units=512, activation='relu'))
model.add(keras.layers.Dense(units=6, activation='softmax'))

model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 148, 148, 32)      2432      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16928)             0         
_________________________________________________________________
high-level-features (Dense)  (None, 2048)              34670592  
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1049088   
__________

In [23]:
# Compile the model
model.compile(optimizer=keras.optimizers.Adamax(), loss='sparse_categorical_crossentropy', metrics=['acc'])

In [24]:
# End training when accuracy stops improving (optional)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)

In [25]:
# Train model

model.fit(X_tr, y_tr, epochs=100, validation_split=0.2, callbacks=[early_stopping], verbose=1, workers=-1)


Train on 336 samples, validate on 84 samples
Epoch 1/100
336/336 [==============================] - 13s 38ms/step - loss: 2.4188 - acc: 0.2292 - val_loss: 2.0835 - val_acc: 0.0000e+00
Epoch 2/100
336/336 [==============================] - 10s 29ms/step - loss: 1.6807 - acc: 0.3690 - val_loss: 1.9241 - val_acc: 0.0595
Epoch 3/100
336/336 [==============================] - 10s 29ms/step - loss: 1.6329 - acc: 0.4762 - val_loss: 2.1994 - val_acc: 0.0476
Epoch 4/100
336/336 [==============================] - 10s 29ms/step - loss: 1.3864 - acc: 0.4881 - val_loss: 1.7872 - val_acc: 0.2143
Epoch 5/100
336/336 [==============================] - 10s 30ms/step - loss: 1.0961 - acc: 0.6518 - val_loss: 1.5148 - val_acc: 0.3571
Epoch 6/100
336/336 [==============================] - 10s 31ms/step - loss: 0.7098 - acc: 0.7738 - val_loss: 1.7244 - val_acc: 0.2738
Epoch 7/100
336/336 [==============================] - 10s 30ms/step - loss: 0.3686 - acc: 0.8958 - val_loss: 1.5563 - val_acc: 0.3690
Epoch 

In [26]:
scores = model.evaluate(X_te, y_te, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 52.00%


In [27]:
print("Feature extraction from the model")
feature_extractor = keras.Model(
   inputs=model.inputs,
   outputs=model.get_layer(name="high-level-features").output,
)


Feature extraction from the model


In [28]:
feature_extractor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2_input (InputLayer)  (None, 299, 299, 3)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 148, 148, 32)      2432      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16928)             0         
_________________________________________________________________
high-level-features (Dense)  (None, 2048)              34670592  
Total para

In [29]:
print("The feature extractor method is used to get my high-level features")

features = feature_extractor.predict(X_te)
features.shape

The feature extractor method is used to get my high-level features


(50, 2048)

In [30]:
accuracy=loadModel('logistic').score(features, y_te)

Loading model from  model-logistic.sav


In [31]:
accuracy=accuracy*100
print("  {} - Accuracy on test dataset: {:.1f}%\n".format('knn', accuracy))

  knn - Accuracy on test dataset: 12.0%



In [40]:
from tensorflow.keras.models import save_model

save_model(model, 'model-cnn.sav')

In [42]:

type(model)

tensorflow.python.keras.engine.sequential.Sequential